In [ ]:
# 🔵 RUN THIS ONLY ONCE

import requests
from bs4 import BeautifulSoup
import pandas as pd
import time


In [ ]:
# 🟢 RUN THIS ONLY ONCE

def extract_rating(star_elements):
    if star_elements:
        filled_stars = star_elements.find_all('span', class_='star fill')
        return len(filled_stars)
    return 'N/A'

def get_rating_by_header(rating_elements, header_text):
    for element in rating_elements:
        header = element.find('td', class_='review-rating-header')
        if header and header_text.lower() in header.text.strip().lower():
            rating_stars = element.find('td', class_='review-rating-stars')
            return extract_rating(rating_stars)
    return 'N/A'

def get_value_by_header(elements, header_text, class_type="review-rating-header"):
    for element in elements:
        header = element.find('td', class_=class_type)
        if header and header_text.lower() in header.text.strip().lower():
            value_td = element.find('td', class_='review-value')
            if value_td:
                return value_td.text.strip()
    return 'N/A'

def scrape_page(page_number, airline_name):
    url = f'https://www.airlinequality.com/airline-reviews/{airline_name}/page/{page_number}/'
    print(f"Scraping page {page_number}: {url}")

    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    reviews = []
    for review_div in soup.find_all('div', class_='body'):
        name_tag = review_div.find('span', itemprop='name')
        date_tag = review_div.find('time', itemprop='datePublished')
        text_content_div = review_div.find('div', class_='text_content')

        name = name_tag.text.strip() if name_tag else 'N/A'
        date_published = date_tag.get('datetime') if date_tag else 'N/A'
        text_content = text_content_div.text.strip() if text_content_div else 'N/A'

        seat_type = 'N/A'
        type_of_traveller = 'N/A'
        route = 'N/A'
        ground_service = 'N/A'
        wifi_connectivity = 'N/A'
        date_flown = 'N/A'

        info_rows = review_div.find_all('tr')
        for row in info_rows:
            header_td = row.find('td', class_='review-rating-header')
            if header_td and 'date flown' in header_td.text.strip().lower():
                value_td = row.find('td', class_='review-value')
                if value_td:
                    date_flown = value_td.text.strip()
                    break

        if date_flown == 'N/A':
            date_flown = get_value_by_header(info_rows, "date flown")

        all_tds = review_div.find_all('td')
        for i in range(len(all_tds)):
            td_class = all_tds[i].get('class')
            if td_class and 'review-rating-header' in td_class:
                header = all_tds[i].text.strip()
                if i + 1 < len(all_tds):
                    next_td = all_tds[i + 1]
                    if header == 'Seat Type':
                        seat_type = next_td.text.strip()
                    elif header == 'Type Of Traveller':
                        type_of_traveller = next_td.text.strip()
                    elif header == 'Route':
                        route = next_td.text.strip()
                    elif header == 'Ground Service':
                        ground_service = extract_rating(next_td)
                    elif header == 'Wifi & Connectivity':
                        wifi_connectivity = extract_rating(next_td)
                    elif 'date flown' in header.lower():
                        date_flown = next_td.text.strip()

        review_stats = review_div.find('div', class_='review-stats')
        rating_elements = review_stats.find_all('tr') if review_stats else []

        seat_comfort = get_rating_by_header(rating_elements, "Seat Comfort")
        cabin_staff_service = get_rating_by_header(rating_elements, "Cabin Staff Service")
        food_beverages = get_rating_by_header(rating_elements, "Food & Beverages")
        inflight_entertainment = get_rating_by_header(rating_elements, "Inflight Entertainment")
        value_for_money = get_rating_by_header(rating_elements, "Value For Money")

        recommended = 'N/A'
        for row in rating_elements:
            header_td = row.find('td', class_='review-rating-header recommended')
            if header_td and 'recommended' in header_td.text.strip().lower():
                value_td = row.find('td', class_='review-value')
                if value_td:
                    if 'rating-yes' in value_td.get('class', []) or value_td.text.strip().lower() == 'yes':
                        recommended = 'Yes'
                    else:
                        recommended = 'No'
                break

        reviews.append({
            'Airline': airline_name,
            'Reviewer Names': name,
            'Review Date': date_published,
            'Month Flown': date_flown,
            'Review Text': text_content,
            'Type of Traveller': type_of_traveller,
            'Seat Type': seat_type,
            'Route': route,
            'Seat Comfort': seat_comfort,
            'Cabin Staff Service': cabin_staff_service,
            'Food & Beverages': food_beverages,
            'Inflight Entertainment': inflight_entertainment,
            'Value For Money': value_for_money,
            'Ground Service': ground_service,
            'Wifi & Connectivity': wifi_connectivity,
            'Recommended': recommended
        })

    return reviews

def scrape_airline_reviews(airline_name, total_pages):
    print(f"\n{'='*50}")
    print(f"SCRAPING {airline_name.upper()}")
    print(f"{'='*50}\n")

    all_reviews = []

    for page in range(1, total_pages + 1):
        try:
            page_reviews = scrape_page(page, airline_name)
            all_reviews.extend(page_reviews)

            if page % 10 == 0 or page == total_pages:
                print(f"{page}/{total_pages} pages extraction completed for {airline_name}")

            if page < total_pages:
                time.sleep(1)

        except Exception as e:
            print(f"Error scraping page {page} for {airline_name}: {e}")

    print(f"Total reviews scraped for {airline_name}: {len(all_reviews)}")

    formatted_name = airline_name.replace('-', '_').lower()
    formatted_csv = f"{formatted_name}_reviews.csv"

    df = pd.DataFrame(all_reviews)
    globals()[f"{formatted_name}_reviews"] = df

    df.to_csv(formatted_csv, index=False)
    print(f"Data saved to {formatted_csv} and DataFrame stored as variable: {formatted_name}_reviews")


1. ADRIA-AIRWAYS

In [ ]:
# 🟡 MODIFY AND RUN THIS FOR EACH AIRLINE

scrape_airline_reviews("adria-airways", 10)

# Example for a different airline:
# scrape_airline_reviews("air-india", 125)



SCRAPING ADRIA-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/adria-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/adria-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/adria-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/adria-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/adria-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/adria-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/adria-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/adria-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/adria-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/adria-airways/page/10/
10/10 pages extraction completed for adria-airways
Total reviews scraped for adria-airways: 91
Data saved to adri

In [ ]:
adria_airways_reviews.head(3)

,Airline,Reviewer Names,Review Date,Month Flown,Review Text,Type of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Ground Service,Wifi & Connectivity,Recommended
0,adria-airways,D Praetextatus,2019-09-28,September 2019,Not Verified | Please do a favor yourself and...,Solo Leisure,Economy Class,Frankfurt to Pristina,1,1,N/A,N/A,1,1,N/A,No
1,adria-airways,D Meijer,2019-09-24,September 2019,✅ Trip Verified | Do not book a flight with th...,Couple Leisure,Economy Class,Sofia to Amsterdam via Ljubljana,1,1,1,1,1,1,1,No
2,adria-airways,Herbse Mayer,2019-09-17,September 2019,✅ Trip Verified | Had very bad experience wit...,Couple Leisure,Economy Class,Sarajevo to Ljubljana,1,1,1,1,1,1,1,No


2. aegean-airlines

In [ ]:
scrape_airline_reviews("aegean-airlines", 82)


SCRAPING AEGEAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/aegean-airlines/page/10/
10/82 pages extraction completed for aegean-airlines
Scraping page 11: https://www.airlineq

3. aer-lingus

In [ ]:
scrape_airline_reviews("aer-lingus", 103)


SCRAPING AER-LINGUS

Scraping page 1: https://www.airlinequality.com/airline-reviews/aer-lingus/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/aer-lingus/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/aer-lingus/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/aer-lingus/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/aer-lingus/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/aer-lingus/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/aer-lingus/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/aer-lingus/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/aer-lingus/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/aer-lingus/page/10/
10/103 pages extraction completed for aer-lingus
Scraping page 11: https://www.airlinequality.com/airline-reviews/aer-lingus/page/11/
Scraping pag

4. aeroflot-russian-airlines

In [ ]:
scrape_airline_reviews("aeroflot-russian-airlines", 60)


SCRAPING AEROFLOT-RUSSIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/aeroflot-russian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/aeroflot-russian

5. aerolineas-argentinas

In [ ]:
scrape_airline_reviews("aerolineas-argentinas", 26)


SCRAPING AEROLINEAS-ARGENTINAS

Scraping page 1: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/aerolineas-argentinas/page/10/
10/26 pages extraction co

6. aeromexico

In [ ]:
scrape_airline_reviews("aeromexico", 74)


SCRAPING AEROMEXICO

Scraping page 1: https://www.airlinequality.com/airline-reviews/aeromexico/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/aeromexico/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/aeromexico/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/aeromexico/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/aeromexico/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/aeromexico/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/aeromexico/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/aeromexico/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/aeromexico/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/aeromexico/page/10/
10/74 pages extraction completed for aeromexico
Scraping page 11: https://www.airlinequality.com/airline-reviews/aeromexico/page/11/
Scraping page

7. air-arabia

In [ ]:
scrape_airline_reviews("air-arabia", 18)


SCRAPING AIR-ARABIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-arabia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-arabia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-arabia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-arabia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-arabia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-arabia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-arabia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-arabia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-arabia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-arabia/page/10/
10/18 pages extraction completed for air-arabia
Scraping page 11: https://www.airlinequality.com/airline-reviews/air-arabia/page/11/
Scraping page

8. air-astana

In [ ]:
scrape_airline_reviews("air-astana", 25)


SCRAPING AIR-ASTANA

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-astana/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-astana/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-astana/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-astana/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-astana/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-astana/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-astana/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-astana/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-astana/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-astana/page/10/
10/25 pages extraction completed for air-astana
Scraping page 11: https://www.airlinequality.com/airline-reviews/air-astana/page/11/
Scraping page

9. air-berlin

In [ ]:
scrape_airline_reviews("air-berlin", 49)


SCRAPING AIR-BERLIN

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-berlin/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-berlin/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-berlin/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-berlin/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-berlin/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-berlin/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-berlin/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-berlin/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-berlin/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-berlin/page/10/
10/49 pages extraction completed for air-berlin
Scraping page 11: https://www.airlinequality.com/airline-reviews/air-berlin/page/11/
Scraping page

10. air-canada-rouge

In [ ]:
scrape_airline_reviews("air-canada-rouge", 124)


SCRAPING AIR-CANADA-ROUGE

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-canada-rouge/page/10/
10/124 pages extraction completed for air-canada-rouge
Scraping page 11: https:/

11. air-china

In [ ]:
scrape_airline_reviews("air-china", 80)


SCRAPING AIR-CHINA

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-china/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-china/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-china/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-china/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-china/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-china/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-china/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-china/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-china/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-china/page/10/
10/80 pages extraction completed for air-china
Scraping page 11: https://www.airlinequality.com/airline-reviews/air-china/page/11/
Scraping page 12: https://

12. air-europa

In [ ]:
scrape_airline_reviews("air-europa", 53)


SCRAPING AIR-EUROPA

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-europa/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-europa/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-europa/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-europa/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-europa/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-europa/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-europa/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-europa/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-europa/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-europa/page/10/
10/53 pages extraction completed for air-europa
Scraping page 11: https://www.airlinequality.com/airline-reviews/air-europa/page/11/
Scraping page

13. air-india-express

In [ ]:
scrape_airline_reviews("air-india-express", 15)


SCRAPING AIR-INDIA-EXPRESS

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-india-express/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-india-express/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-india-express/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-india-express/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-india-express/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-india-express/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-india-express/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-india-express/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-india-express/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-india-express/page/10/
10/15 pages extraction completed for air-india-express
Scraping page 

14. air-malta

In [ ]:
scrape_airline_reviews("air-malta", 19)


SCRAPING AIR-MALTA

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-malta/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-malta/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-malta/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-malta/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-malta/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-malta/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-malta/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-malta/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-malta/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-malta/page/10/
10/19 pages extraction completed for air-malta
Scraping page 11: https://www.airlinequality.com/airline-reviews/air-malta/page/11/
Scraping page 12: https://

15. air-mauritius

In [ ]:
scrape_airline_reviews("air-mauritius", 22)


SCRAPING AIR-MAURITIUS

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-mauritius/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-mauritius/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-mauritius/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-mauritius/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-mauritius/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-mauritius/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-mauritius/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-mauritius/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-mauritius/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-mauritius/page/10/
10/22 pages extraction completed for air-mauritius
Scraping page 11: https://www.airlinequality.com/airline-revie

16. air-new-zealand

In [ ]:
scrape_airline_reviews("air-new-zealand", 88)


SCRAPING AIR-NEW-ZEALAND

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/10/
10/88 pages extraction completed for air-new-zealand
Scraping page 11: https://www.airlineq

17. air-serbia

In [ ]:
scrape_airline_reviews("air-new-zealand", 32)


SCRAPING AIR-NEW-ZEALAND

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-new-zealand/page/10/
10/32 pages extraction completed for air-new-zealand
Scraping page 11: https://www.airlineq

18. air-tahiti-nui

In [ ]:
scrape_airline_reviews("air-tahiti-nui", 12)


SCRAPING AIR-TAHITI-NUI

Scraping page 1: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/air-tahiti-nui/page/10/
10/12 pages extraction completed for air-tahiti-nui
Scraping page 11: https://www.airlinequality.com/a

19. airasia

In [ ]:
scrape_airline_reviews("airasia", 99)


SCRAPING AIRASIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/airasia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/airasia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/airasia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/airasia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/airasia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/airasia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/airasia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/airasia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/airasia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/airasia/page/10/
10/99 pages extraction completed for airasia
Scraping page 11: https://www.airlinequality.com/airline-reviews/airasia/page/11/
Scraping page 12: https://www.airlinequality.com/air

20. airasia-india

In [ ]:
scrape_airline_reviews("airasia-india", 13)


SCRAPING AIRASIA-INDIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/airasia-india/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/airasia-india/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/airasia-india/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/airasia-india/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/airasia-india/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/airasia-india/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/airasia-india/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/airasia-india/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/airasia-india/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/airasia-india/page/10/
10/13 pages extraction completed for airasia-india
Scraping page 11: https://www.airlinequality.com/airline-revie

21. airasia-x

In [ ]:
scrape_airline_reviews("airasia-x", 40)


SCRAPING AIRASIA-X

Scraping page 1: https://www.airlinequality.com/airline-reviews/airasia-x/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/airasia-x/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/airasia-x/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/airasia-x/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/airasia-x/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/airasia-x/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/airasia-x/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/airasia-x/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/airasia-x/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/airasia-x/page/10/
10/40 pages extraction completed for airasia-x
Scraping page 11: https://www.airlinequality.com/airline-reviews/airasia-x/page/11/
Scraping page 12: https://

22. airbaltic

In [ ]:
scrape_airline_reviews("airbaltic", 28)


SCRAPING AIRBALTIC

Scraping page 1: https://www.airlinequality.com/airline-reviews/airbaltic/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/airbaltic/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/airbaltic/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/airbaltic/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/airbaltic/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/airbaltic/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/airbaltic/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/airbaltic/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/airbaltic/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/airbaltic/page/10/
10/28 pages extraction completed for airbaltic
Scraping page 11: https://www.airlinequality.com/airline-reviews/airbaltic/page/11/
Scraping page 12: https://

23. akasa-air

In [ ]:
scrape_airline_reviews("akasa-air", 19)


SCRAPING AKASA-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/akasa-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/akasa-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/akasa-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/akasa-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/akasa-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/akasa-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/akasa-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/akasa-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/akasa-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/akasa-air/page/10/
10/19 pages extraction completed for akasa-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/akasa-air/page/11/
Scraping page 12: https://

24. alaska-airlines

In [ ]:
scrape_airline_reviews("alaska-airlines", 103)


SCRAPING ALASKA-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/alaska-airlines/page/10/
10/103 pages extraction completed for alaska-airlines
Scraping page 11: https://www.airline

25. alitalia

In [ ]:
scrape_airline_reviews("alitalia", 70)


SCRAPING ALITALIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/alitalia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/alitalia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/alitalia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/alitalia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/alitalia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/alitalia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/alitalia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/alitalia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/alitalia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/alitalia/page/10/
10/70 pages extraction completed for alitalia
Scraping page 11: https://www.airlinequality.com/airline-reviews/alitalia/page/11/
Scraping page 12: https://www.airlinequ

26. allegiant-air

In [ ]:
scrape_airline_reviews("allegiant-air", 200)


SCRAPING ALLEGIANT-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/allegiant-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/allegiant-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/allegiant-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/allegiant-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/allegiant-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/allegiant-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/allegiant-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/allegiant-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/allegiant-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/allegiant-air/page/10/
10/200 pages extraction completed for allegiant-air
Scraping page 11: https://www.airlinequality.com/airline-revi

27. american-eagle

In [ ]:
scrape_airline_reviews("american-eagle", 18)


SCRAPING AMERICAN-EAGLE

Scraping page 1: https://www.airlinequality.com/airline-reviews/american-eagle/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/american-eagle/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/american-eagle/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/american-eagle/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/american-eagle/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/american-eagle/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/american-eagle/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/american-eagle/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/american-eagle/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/american-eagle/page/10/
10/18 pages extraction completed for american-eagle
Scraping page 11: https://www.airlinequality.com/a

28. ana-all-nippon-airways

In [ ]:
scrape_airline_reviews("ana-all-nippon-airways", 61)


SCRAPING ANA-ALL-NIPPON-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/ana-all-nippon-airways/page/10/
10/61 pages ex

29. asiana-airlines

In [ ]:
scrape_airline_reviews("asiana-airlines", 52)


SCRAPING ASIANA-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/asiana-airlines/page/10/
10/52 pages extraction completed for asiana-airlines
Scraping page 11: https://www.airlineq

30. austrian-airlines

In [ ]:
scrape_airline_reviews("austrian-airlines", 79)


SCRAPING AUSTRIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/austrian-airlines/page/10/
10/79 pages extraction completed for austrian-airlines
Scraping page 

31. avelo-airlines

In [ ]:
scrape_airline_reviews("avelo-airlines", 17)


SCRAPING AVELO-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/avelo-airlines/page/10/
10/17 pages extraction completed for avelo-airlines
Scraping page 11: https://www.airlinequality.com/a

32. avianca

In [ ]:
scrape_airline_reviews("avianca", 77)


SCRAPING AVIANCA

Scraping page 1: https://www.airlinequality.com/airline-reviews/avianca/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/avianca/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/avianca/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/avianca/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/avianca/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/avianca/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/avianca/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/avianca/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/avianca/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/avianca/page/10/
10/77 pages extraction completed for avianca
Scraping page 11: https://www.airlinequality.com/airline-reviews/avianca/page/11/
Scraping page 12: https://www.airlinequality.com/air

33. azul-linhas-aereas-brasileiras

In [ ]:
scrape_airline_reviews("azul-linhas-aereas-brasileiras", 11)


SCRAPING AZUL-LINHAS-AEREAS-BRASILEIRAS

Scraping page 1: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/azul-linhas-aereas-brasileiras/page/9/
Scraping page 10: https://www.a

In [ ]:
A_airlines_review = pd.concat([
    adria_airways_reviews,
    aegean_airlines_reviews,
    aer_lingus_reviews,
    aeroflot_russian_airlines_reviews,
    aerolineas_argentinas_reviews,
    aeromexico_reviews,
    air_arabia_reviews,
    air_astana_reviews,
    air_berlin_reviews,
    air_canada_rouge_reviews,
    air_china_reviews,
    air_europa_reviews,
    air_india_express_reviews,
    air_malta_reviews,
    air_mauritius_reviews,
    air_new_zealand_reviews,
    air_tahiti_nui_reviews,
    airasia_reviews,
    airasia_india_reviews,
    airasia_x_reviews,
    airbaltic_reviews,
    akasa_air_reviews,
    alaska_airlines_reviews,
    alitalia_reviews,
    allegiant_air_reviews,
    american_eagle_reviews,
    ana_all_nippon_airways_reviews,
    asiana_airlines_reviews,
    austrian_airlines_reviews,
    avelo_airlines_reviews,
    avianca_reviews,
    azul_linhas_aereas_brasileiras_reviews

], axis=0, ignore_index=True)

In [ ]:
A_airlines_review.shape

(16763, 16)

In [ ]:
A_airlines_review.to_csv('A_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH B**

34. bangkok-airways

In [ ]:
scrape_airline_reviews("bangkok-airways", 42)


SCRAPING BANGKOK-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/bangkok-airways/page/10/
10/42 pages extraction completed for bangkok-airways
Scraping page 11: https://www.airlineq

35. batik-air

In [ ]:
scrape_airline_reviews("batik-air", 21)


SCRAPING BATIK-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/batik-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/batik-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/batik-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/batik-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/batik-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/batik-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/batik-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/batik-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/batik-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/batik-air/page/10/
10/21 pages extraction completed for batik-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/batik-air/page/11/
Scraping page 12: https://

36. biman-bangladesh

In [ ]:
scrape_airline_reviews("biman-bangladesh", 11)


SCRAPING BIMAN-BANGLADESH

Scraping page 1: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/biman-bangladesh/page/10/
10/11 pages extraction completed for biman-bangladesh
Scraping page 11: https://

37. blue-air

In [ ]:
scrape_airline_reviews("blue-air", 15)


SCRAPING BLUE-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/blue-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/blue-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/blue-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/blue-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/blue-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/blue-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/blue-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/blue-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/blue-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/blue-air/page/10/
10/15 pages extraction completed for blue-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/blue-air/page/11/
Scraping page 12: https://www.airlinequ

38. breeze-airways

In [ ]:
scrape_airline_reviews("breeze-airways", 39)


SCRAPING BREEZE-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/breeze-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/breeze-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/breeze-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/breeze-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/breeze-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/breeze-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/breeze-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/breeze-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/breeze-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/breeze-airways/page/10/
10/39 pages extraction completed for breeze-airways
Scraping page 11: https://www.airlinequality.com/a

39. brussels-airlines

In [ ]:
scrape_airline_reviews("brussels-airlines", 58)


SCRAPING BRUSSELS-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/brussels-airlines/page/10/
10/58 pages extraction completed for brussels-airlines
Scraping page 

40. bulgaria-air

In [ ]:
scrape_airline_reviews("bulgaria-air", 12)


SCRAPING BULGARIA-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/bulgaria-air/page/10/
10/12 pages extraction completed for bulgaria-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/bulgaria-

In [ ]:
B_airlines_review = pd.concat([
    bangkok_airways_reviews,
    batik_air_reviews,
    biman_bangladesh_reviews,
    blue_air_reviews,
    breeze_airways_reviews,
    brussels_airlines_reviews,
    bulgaria_air_reviews
], axis=0, ignore_index=True)

In [ ]:
B_airlines_review.shape

(1952, 16)

In [ ]:
B_airlines_review.to_csv('B_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH C**

41. caribbean-airlines

In [ ]:
scrape_airline_reviews("caribbean-airlines", 12)


SCRAPING CARIBBEAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/caribbean-airlines/page/10/
10/12 pages extraction completed for caribbean-airlines
Sc

42. cathay-pacific-airways

In [ ]:
scrape_airline_reviews("cathay-pacific-airways", 149)


SCRAPING CATHAY-PACIFIC-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/cathay-pacific-airways/page/10/
10/149 pages e

43. cebu-pacific

In [ ]:
scrape_airline_reviews("cebu-pacific", 43)


SCRAPING CEBU-PACIFIC

Scraping page 1: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/cebu-pacific/page/10/
10/43 pages extraction completed for cebu-pacific
Scraping page 11: https://www.airlinequality.com/airline-reviews/cebu-paci

44. china-airlines

In [ ]:
scrape_airline_reviews("china-airlines", 49)


SCRAPING CHINA-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/china-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/china-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/china-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/china-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/china-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/china-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/china-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/china-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/china-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/china-airlines/page/10/
10/49 pages extraction completed for china-airlines
Scraping page 11: https://www.airlinequality.com/a

45. cityjet

In [ ]:
scrape_airline_reviews("cityjet", 10)


SCRAPING CITYJET

Scraping page 1: https://www.airlinequality.com/airline-reviews/cityjet/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/cityjet/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/cityjet/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/cityjet/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/cityjet/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/cityjet/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/cityjet/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/cityjet/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/cityjet/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/cityjet/page/10/
10/10 pages extraction completed for cityjet
Total reviews scraped for cityjet: 100
Data saved to cityjet_reviews.csv and DataFrame stored as variable: cityjet_reviews


46. condor-airlines

In [ ]:
scrape_airline_reviews("condor-airlines", 49)


SCRAPING CONDOR-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/condor-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/condor-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/condor-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/condor-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/condor-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/condor-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/condor-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/condor-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/condor-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/condor-airlines/page/10/
10/49 pages extraction completed for condor-airlines
Scraping page 11: https://www.airlineq

47. copa-airlines

In [ ]:
scrape_airline_reviews("copa-airlines", 44)


SCRAPING COPA-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/copa-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/copa-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/copa-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/copa-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/copa-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/copa-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/copa-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/copa-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/copa-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/copa-airlines/page/10/
10/44 pages extraction completed for copa-airlines
Scraping page 11: https://www.airlinequality.com/airline-revie

48. csa-czech-airlines

In [ ]:
scrape_airline_reviews("csa-czech-airlines", 23)


SCRAPING CSA-CZECH-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/csa-czech-airlines/page/10/
10/23 pages extraction completed for csa-czech-airlines
Sc

49. cubana-airlines

In [ ]:
scrape_airline_reviews("cubana-airlines", 11)


SCRAPING CUBANA-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/cubana-airlines/page/10/
10/11 pages extraction completed for cubana-airlines
Scraping page 11: https://www.airlineq

In [ ]:
C_airlines_review = pd.concat([
  caribbean_airlines_reviews,
  cathay_pacific_airways_reviews,
  cebu_pacific_reviews,
  china_airlines_reviews,
  cityjet_reviews,
  condor_airlines_reviews,
  copa_airlines_reviews,
  csa_czech_airlines_reviews,
  cubana_airlines_reviews
], axis=0, ignore_index=True)

In [ ]:
C_airlines_review.shape

(3863, 16)

In [ ]:
C_airlines_review.to_csv('C_airlines_review.csv', index=False)

# AIRLINES STARTING WITH D

50. dragonair

In [ ]:
scrape_airline_reviews("dragonair", 18)


SCRAPING DRAGONAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/dragonair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/dragonair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/dragonair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/dragonair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/dragonair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/dragonair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/dragonair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/dragonair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/dragonair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/dragonair/page/10/
10/18 pages extraction completed for dragonair
Scraping page 11: https://www.airlinequality.com/airline-reviews/dragonair/page/11/
Scraping page 12: https://

51.discover-airlines

In [ ]:
scrape_airline_reviews("discover-airlines", 6)


SCRAPING DISCOVER-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/discover-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/discover-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/discover-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/discover-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/discover-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/discover-airlines/page/6/
6/6 pages extraction completed for discover-airlines
Total reviews scraped for discover-airlines: 54
Data saved to discover_airlines_reviews.csv and DataFrame stored as variable: discover_airlines_reviews


In [ ]:
D_airlines_review = pd.concat([
  dragonair_reviews,
  discover_airlines_reviews
], axis=0, ignore_index=True)

In [ ]:
D_airlines_review.shape

(228, 16)

In [ ]:
D_airlines_review.to_csv('D_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH E**

52. edelweiss-air

In [ ]:
scrape_airline_reviews("edelweiss-air", 12)


SCRAPING EDELWEISS-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/edelweiss-air/page/10/
10/12 pages extraction completed for edelweiss-air
Scraping page 11: https://www.airlinequality.com/airline-revie

53. egyptair

In [ ]:
scrape_airline_reviews("egyptair", 51)


SCRAPING EGYPTAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/egyptair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/egyptair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/egyptair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/egyptair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/egyptair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/egyptair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/egyptair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/egyptair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/egyptair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/egyptair/page/10/
10/51 pages extraction completed for egyptair
Scraping page 11: https://www.airlinequality.com/airline-reviews/egyptair/page/11/
Scraping page 12: https://www.airlinequ

54. el-al-israel-airlines

In [ ]:
scrape_airline_reviews("el-al-israel-airlines", 31)


SCRAPING EL-AL-ISRAEL-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/el-al-israel-airlines/page/10/
10/31 pages extraction co

55. ethiopian-airlines

In [ ]:
scrape_airline_reviews("ethiopian-airlines", 64)


SCRAPING ETHIOPIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/ethiopian-airlines/page/10/
10/64 pages extraction completed for ethiopian-airlines
Sc

56. eva-air

In [ ]:
scrape_airline_reviews("eva-air", 66)


SCRAPING EVA-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/eva-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/eva-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/eva-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/eva-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/eva-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/eva-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/eva-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/eva-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/eva-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/eva-air/page/10/
10/66 pages extraction completed for eva-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/eva-air/page/11/
Scraping page 12: https://www.airlinequality.com/air

57. eurowings

In [ ]:
scrape_airline_reviews("eurowings", 48)


SCRAPING EUROWINGS

Scraping page 1: https://www.airlinequality.com/airline-reviews/eurowings/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/eurowings/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/eurowings/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/eurowings/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/eurowings/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/eurowings/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/eurowings/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/eurowings/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/eurowings/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/eurowings/page/10/
10/48 pages extraction completed for eurowings
Scraping page 11: https://www.airlinequality.com/airline-reviews/eurowings/page/11/
Scraping page 12: https://

In [ ]:
E_airlines_review = pd.concat([
  edelweiss_air_reviews,
  egyptair_reviews,
  el_al_israel_airlines_reviews,
  ethiopian_airlines_reviews,
  eva_air_reviews,
  eurowings_reviews
], axis=0, ignore_index=True)

In [ ]:
E_airlines_review.shape

(2685, 16)

In [ ]:
E_airlines_review.to_csv('E_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH F**

58. fiji-airways

In [ ]:
scrape_airline_reviews("fiji-airways", 33)


SCRAPING FIJI-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/fiji-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/fiji-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/fiji-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/fiji-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/fiji-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/fiji-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/fiji-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/fiji-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/fiji-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/fiji-airways/page/10/
10/33 pages extraction completed for fiji-airways
Scraping page 11: https://www.airlinequality.com/airline-reviews/fiji-airw

59. finnair


In [ ]:
scrape_airline_reviews("finnair", 96)


SCRAPING FINNAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/finnair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/finnair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/finnair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/finnair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/finnair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/finnair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/finnair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/finnair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/finnair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/finnair/page/10/
10/96 pages extraction completed for finnair
Scraping page 11: https://www.airlinequality.com/airline-reviews/finnair/page/11/
Scraping page 12: https://www.airlinequality.com/air

60. flair-airlines

In [ ]:
scrape_airline_reviews("flair-airlines", 115)


SCRAPING FLAIR-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/flair-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/flair-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/flair-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/flair-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/flair-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/flair-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/flair-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/flair-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/flair-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/flair-airlines/page/10/
10/115 pages extraction completed for flair-airlines
Scraping page 11: https://www.airlinequality.com/

61. firefly

In [ ]:
scrape_airline_reviews("firefly", 11)


SCRAPING FIREFLY

Scraping page 1: https://www.airlinequality.com/airline-reviews/firefly/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/firefly/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/firefly/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/firefly/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/firefly/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/firefly/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/firefly/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/firefly/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/firefly/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/firefly/page/10/
10/11 pages extraction completed for firefly
Scraping page 11: https://www.airlinequality.com/airline-reviews/firefly/page/11/
11/11 pages extraction completed for firefly
Total r

In [ ]:
F_airlines_review = pd.concat([
  fiji_airways_reviews,
  finnair_reviews,
  flair_airlines_reviews,
  firefly_reviews
], axis=0, ignore_index=True)

In [ ]:
F_airlines_review.shape

(2526, 16)

In [ ]:
F_airlines_review.to_csv('F_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH G**

62. garuda-indonesia

In [ ]:
scrape_airline_reviews("garuda-indonesia", 95)


SCRAPING GARUDA-INDONESIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/garuda-indonesia/page/10/
10/95 pages extraction completed for garuda-indonesia
Scraping page 11: https://

63. germanwings

In [ ]:
scrape_airline_reviews("germanwings", 15)


SCRAPING GERMANWINGS

Scraping page 1: https://www.airlinequality.com/airline-reviews/germanwings/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/germanwings/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/germanwings/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/germanwings/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/germanwings/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/germanwings/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/germanwings/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/germanwings/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/germanwings/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/germanwings/page/10/
10/15 pages extraction completed for germanwings
Scraping page 11: https://www.airlinequality.com/airline-reviews/germanwings/page/11/


64. go-first

In [ ]:
scrape_airline_reviews("go-first", 20)


SCRAPING GO-FIRST

Scraping page 1: https://www.airlinequality.com/airline-reviews/go-first/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/go-first/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/go-first/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/go-first/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/go-first/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/go-first/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/go-first/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/go-first/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/go-first/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/go-first/page/10/
10/20 pages extraction completed for go-first
Scraping page 11: https://www.airlinequality.com/airline-reviews/go-first/page/11/
Scraping page 12: https://www.airlinequ

65. goair

In [ ]:
scrape_airline_reviews("goair", 23)


SCRAPING GOAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/goair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/goair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/goair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/goair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/goair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/goair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/goair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/goair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/goair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/goair/page/10/
10/23 pages extraction completed for goair
Scraping page 11: https://www.airlinequality.com/airline-reviews/goair/page/11/
Scraping page 12: https://www.airlinequality.com/airline-reviews/goair/page/12

66. gulf-air

In [ ]:
scrape_airline_reviews("gulf-air", 43)


SCRAPING GULF-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/gulf-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/gulf-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/gulf-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/gulf-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/gulf-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/gulf-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/gulf-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/gulf-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/gulf-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/gulf-air/page/10/
10/43 pages extraction completed for gulf-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/gulf-air/page/11/
Scraping page 12: https://www.airlinequ

In [ ]:
G_airlines_review = pd.concat([
  garuda_indonesia_reviews,
  germanwings_reviews,
  go_first_reviews,
  goair_reviews,
  gulf_air_reviews
], axis=0, ignore_index=True)

In [ ]:
G_airlines_review.shape

(1932, 16)

In [ ]:
G_airlines_review.to_csv('G_airlines_review.csv', index=False)

#**AIRLINES STARTING WITH H**

67. hawaiian-airlines

In [ ]:
scrape_airline_reviews("hawaiian-airlines", 57)


SCRAPING HAWAIIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/hawaiian-airlines/page/10/
10/57 pages extraction completed for hawaiian-airlines
Scraping page 

68. hainan-airlines

In [ ]:
scrape_airline_reviews("hainan-airlines", 42)


SCRAPING HAINAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/hainan-airlines/page/10/
10/42 pages extraction completed for hainan-airlines
Scraping page 11: https://www.airlineq

69. hong-kong-airlines

In [ ]:
scrape_airline_reviews("hong-kong-airlines", 16)


SCRAPING HONG-KONG-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/hong-kong-airlines/page/10/
10/16 pages extraction completed for hong-kong-airlines
Sc

70. hong-kong

In [ ]:
scrape_airline_reviews("hong-kong", 10)


SCRAPING HONG-KONG

Scraping page 1: https://www.airlinequality.com/airline-reviews/hong-kong/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/hong-kong/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/hong-kong/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/hong-kong/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/hong-kong/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/hong-kong/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/hong-kong/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/hong-kong/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/hong-kong/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/hong-kong/page/10/
10/10 pages extraction completed for hong-kong
Total reviews scraped for hong-kong: 100
Data saved to hong_kong_reviews.csv and DataFrame stored as variable:

In [ ]:
H_airlines_review = pd.concat([
  hainan_airlines_reviews,
  hong_kong_airlines_reviews,
  hong_kong_reviews,
  hawaiian_airlines_reviews
], axis=0, ignore_index=True)

In [ ]:
H_airlines_review.shape

(1231, 16)

In [ ]:
H_airlines_review.to_csv('H_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH I**

71. iberia

In [ ]:
scrape_airline_reviews("iberia", 93)


SCRAPING IBERIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/iberia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/iberia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/iberia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/iberia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/iberia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/iberia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/iberia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/iberia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/iberia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/iberia/page/10/
10/93 pages extraction completed for iberia
Scraping page 11: https://www.airlinequality.com/airline-reviews/iberia/page/11/
Scraping page 12: https://www.airlinequality.com/airline-reviews/

72. icelandair

In [ ]:
scrape_airline_reviews("icelandair", 74)


SCRAPING ICELANDAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/icelandair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/icelandair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/icelandair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/icelandair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/icelandair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/icelandair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/icelandair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/icelandair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/icelandair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/icelandair/page/10/
10/74 pages extraction completed for icelandair
Scraping page 11: https://www.airlinequality.com/airline-reviews/icelandair/page/11/
Scraping page

73. indigo-airlines

In [ ]:
scrape_airline_reviews("indigo-airlines", 56)


SCRAPING INDIGO-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/indigo-airlines/page/10/
10/56 pages extraction completed for indigo-airlines
Scraping page 11: https://www.airlineq

74. interjet

In [ ]:
scrape_airline_reviews("interjet", 22)


SCRAPING INTERJET

Scraping page 1: https://www.airlinequality.com/airline-reviews/interjet/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/interjet/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/interjet/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/interjet/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/interjet/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/interjet/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/interjet/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/interjet/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/interjet/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/interjet/page/10/
10/22 pages extraction completed for interjet
Scraping page 11: https://www.airlinequality.com/airline-reviews/interjet/page/11/
Scraping page 12: https://www.airlinequ

75. ita-airways

In [ ]:
scrape_airline_reviews("ita-airways", 26)


SCRAPING ITA-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/ita-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/ita-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/ita-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/ita-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/ita-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/ita-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/ita-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/ita-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/ita-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/ita-airways/page/10/
10/26 pages extraction completed for ita-airways
Scraping page 11: https://www.airlinequality.com/airline-reviews/ita-airways/page/11/


In [ ]:
I_airlines_review = pd.concat([
  iberia_reviews,
  icelandair_reviews,
  indigo_airlines_reviews,
  interjet_reviews,
  ita_airways_reviews
], axis=0, ignore_index=True)

In [ ]:
I_airlines_review.shape

(2693, 16)

In [ ]:
I_airlines_review.to_csv('I_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH J**

76. japan-airlines

In [ ]:
scrape_airline_reviews("japan-airlines", 43)


SCRAPING JAPAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/japan-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/japan-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/japan-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/japan-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/japan-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/japan-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/japan-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/japan-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/japan-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/japan-airlines/page/10/
10/43 pages extraction completed for japan-airlines
Scraping page 11: https://www.airlinequality.com/a

77. jet2-com

In [ ]:
scrape_airline_reviews("jet2-com", 59)


SCRAPING JET2-COM

Scraping page 1: https://www.airlinequality.com/airline-reviews/jet2-com/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/jet2-com/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/jet2-com/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/jet2-com/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/jet2-com/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/jet2-com/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/jet2-com/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/jet2-com/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/jet2-com/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/jet2-com/page/10/
10/59 pages extraction completed for jet2-com
Scraping page 11: https://www.airlinequality.com/airline-reviews/jet2-com/page/11/
Scraping page 12: https://www.airlinequ

78. jetblue-airways

In [ ]:
scrape_airline_reviews("jetblue-airways", 167)


SCRAPING JETBLUE-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/jetblue-airways/page/10/
10/167 pages extraction completed for jetblue-airways
Scraping page 11: https://www.airline

79. jetstar-airways

In [ ]:
scrape_airline_reviews("jetstar-airways", 134)


SCRAPING JETSTAR-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/jetstar-airways/page/10/
10/134 pages extraction completed for jetstar-airways
Scraping page 11: https://www.airline

80. jetstar-asia

In [ ]:
scrape_airline_reviews("jetstar-asia", 21)


SCRAPING JETSTAR-ASIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/jetstar-asia/page/10/
10/21 pages extraction completed for jetstar-asia
Scraping page 11: https://www.airlinequality.com/airline-reviews/jetstar-a

81. jetstar-pacific

In [ ]:
scrape_airline_reviews("jetstar-pacific", 12)


SCRAPING JETSTAR-PACIFIC

Scraping page 1: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/jetstar-pacific/page/10/
10/12 pages extraction completed for jetstar-pacific
Scraping page 11: https://www.airlineq

In [ ]:
J_airlines_review = pd.concat([
 japan_airlines_reviews,
 jet2_com_reviews,
 jetblue_airways_reviews,
 jetstar_airways_reviews,
 jetstar_asia_reviews,
 jetstar_pacific_reviews
], axis=0, ignore_index=True)

In [ ]:
J_airlines_review.shape

(4334, 16)

In [ ]:
J_airlines_review.to_csv('J_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH K**

82. kenya-airways

In [ ]:
scrape_airline_reviews("kenya-airways", 51)


SCRAPING KENYA-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/kenya-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/kenya-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/kenya-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/kenya-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/kenya-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/kenya-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/kenya-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/kenya-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/kenya-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/kenya-airways/page/10/
10/51 pages extraction completed for kenya-airways
Scraping page 11: https://www.airlinequality.com/airline-revie

83. korean-air

In [ ]:
scrape_airline_reviews("korean-air", 59)


SCRAPING KOREAN-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/korean-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/korean-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/korean-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/korean-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/korean-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/korean-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/korean-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/korean-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/korean-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/korean-air/page/10/
10/59 pages extraction completed for korean-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/korean-air/page/11/
Scraping page

84. kuwait-airways

In [ ]:
scrape_airline_reviews("kuwait-airways", 33)


SCRAPING KUWAIT-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/kuwait-airways/page/10/
10/33 pages extraction completed for kuwait-airways
Scraping page 11: https://www.airlinequality.com/a

In [ ]:
K_airlines_review = pd.concat([
 kenya_airways_reviews,
 korean_air_reviews,
 kuwait_airways_reviews
], axis=0, ignore_index=True)

In [ ]:
K_airlines_review.shape

(1415, 16)

In [ ]:
K_airlines_review.to_csv('K_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH L**

85. lan-airlines

In [ ]:
scrape_airline_reviews("lan-airlines", 33)


SCRAPING LAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/lan-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/lan-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/lan-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/lan-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/lan-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/lan-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/lan-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/lan-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/lan-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/lan-airlines/page/10/
10/33 pages extraction completed for lan-airlines
Scraping page 11: https://www.airlinequality.com/airline-reviews/lan-airli

86. latam-airlines

In [ ]:
scrape_airline_reviews("latam-airlines", 51)


SCRAPING LATAM-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/latam-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/latam-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/latam-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/latam-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/latam-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/latam-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/latam-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/latam-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/latam-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/latam-airlines/page/10/
10/51 pages extraction completed for latam-airlines
Scraping page 11: https://www.airlinequality.com/a

87. liat

In [ ]:
scrape_airline_reviews("liat", 12)


SCRAPING LIAT

Scraping page 1: https://www.airlinequality.com/airline-reviews/liat/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/liat/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/liat/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/liat/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/liat/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/liat/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/liat/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/liat/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/liat/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/liat/page/10/
10/12 pages extraction completed for liat
Scraping page 11: https://www.airlinequality.com/airline-reviews/liat/page/11/
Scraping page 12: https://www.airlinequality.com/airline-reviews/liat/page/12/
12/12 pages 

88. lion-air

In [ ]:
scrape_airline_reviews("lion-air", 22)


SCRAPING LION-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/lion-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/lion-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/lion-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/lion-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/lion-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/lion-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/lion-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/lion-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/lion-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/lion-air/page/10/
10/22 pages extraction completed for lion-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/lion-air/page/11/
Scraping page 12: https://www.airlinequ

89. lot-polish-airlines

In [ ]:
scrape_airline_reviews("lot-polish-airlines", 74)


SCRAPING LOT-POLISH-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/lot-polish-airlines/page/10/
10/74 pages extraction completed for lot-polish

90. lynx-air

In [ ]:
scrape_airline_reviews("lynx-air", 46)


SCRAPING LYNX-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/lynx-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/lynx-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/lynx-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/lynx-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/lynx-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/lynx-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/lynx-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/lynx-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/lynx-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/lynx-air/page/10/
10/46 pages extraction completed for lynx-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/lynx-air/page/11/
Scraping page 12: https://www.airlinequ

In [ ]:
L_airlines_review = pd.concat([
  lan_airlines_reviews,
  latam_airlines_reviews,
  liat_reviews,
  lion_air_reviews,
  lot_polish_airlines_reviews,
  lynx_air_reviews
], axis=0, ignore_index=True)

In [ ]:
L_airlines_review.shape

(2360, 16)

In [ ]:
L_airlines_review.to_csv('L_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH M**

91. malaysia-airlines

In [ ]:
scrape_airline_reviews("malaysia-airlines", 148)


SCRAPING MALAYSIA-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/malaysia-airlines/page/10/
10/148 pages extraction completed for malaysia-airlines
Scraping page

92. malindo-air

In [ ]:
scrape_airline_reviews("malindo-air", 36)


SCRAPING MALINDO-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/malindo-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/malindo-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/malindo-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/malindo-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/malindo-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/malindo-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/malindo-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/malindo-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/malindo-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/malindo-air/page/10/
10/36 pages extraction completed for malindo-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/malindo-air/page/11/


93. mango

In [ ]:
scrape_airline_reviews("mango", 12)


SCRAPING MANGO

Scraping page 1: https://www.airlinequality.com/airline-reviews/mango/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/mango/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/mango/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/mango/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/mango/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/mango/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/mango/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/mango/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/mango/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/mango/page/10/
10/12 pages extraction completed for mango
Scraping page 11: https://www.airlinequality.com/airline-reviews/mango/page/11/
Scraping page 12: https://www.airlinequality.com/airline-reviews/mango/page/12

94. meridiana

In [ ]:
scrape_airline_reviews("meridiana", 12)


SCRAPING MERIDIANA

Scraping page 1: https://www.airlinequality.com/airline-reviews/meridiana/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/meridiana/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/meridiana/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/meridiana/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/meridiana/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/meridiana/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/meridiana/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/meridiana/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/meridiana/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/meridiana/page/10/
10/12 pages extraction completed for meridiana
Scraping page 11: https://www.airlinequality.com/airline-reviews/meridiana/page/11/
Scraping page 12: https://

95. mea-middle-east-airlines

In [ ]:
scrape_airline_reviews("mea-middle-east-airlines", 12)


SCRAPING MEA-MIDDLE-EAST-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/mea-middle-east-airlines/p

96. monarch-airlines

In [ ]:
scrape_airline_reviews("monarch-airlines", 46)


SCRAPING MONARCH-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/monarch-airlines/page/10/
10/46 pages extraction completed for monarch-airlines
Scraping page 11: https://

In [ ]:
M_airlines_review = pd.concat([
 malaysia_airlines_reviews,
 malindo_air_reviews,
 mango_reviews,
 meridiana_reviews,
 mea_middle_east_airlines_reviews,
 monarch_airlines_reviews
], axis=0, ignore_index=True)

In [ ]:
M_airlines_review.shape

(2619, 16)

In [ ]:
M_airlines_review.to_csv('M_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH N**

97. nok-air

In [ ]:
scrape_airline_reviews("nok-air", 17)


SCRAPING NOK-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/nok-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/nok-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/nok-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/nok-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/nok-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/nok-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/nok-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/nok-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/nok-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/nok-air/page/10/
10/17 pages extraction completed for nok-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/nok-air/page/11/
Scraping page 12: https://www.airlinequality.com/air

98. norse-atlantic-airways

In [ ]:
scrape_airline_reviews("norse-atlantic-airways", 22)


SCRAPING NORSE-ATLANTIC-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/norse-atlantic-airways/page/10/
10/22 pages ex

99. norwegian

In [ ]:
scrape_airline_reviews("norwegian", 133)


SCRAPING NORWEGIAN

Scraping page 1: https://www.airlinequality.com/airline-reviews/norwegian/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/norwegian/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/norwegian/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/norwegian/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/norwegian/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/norwegian/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/norwegian/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/norwegian/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/norwegian/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/norwegian/page/10/
10/133 pages extraction completed for norwegian
Scraping page 11: https://www.airlinequality.com/airline-reviews/norwegian/page/11/
Scraping page 12: https:/

In [ ]:
N_airlines_review = pd.concat([
 nok_air_reviews,
 norse_atlantic_airways_reviews,
 norwegian_reviews
], axis=0, ignore_index=True)

In [ ]:
N_airlines_review.shape

(1703, 16)

In [ ]:
N_airlines_review.to_csv('N_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH O**

100. oman-air

In [ ]:
scrape_airline_reviews("oman-air", 50)


SCRAPING OMAN-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/oman-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/oman-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/oman-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/oman-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/oman-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/oman-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/oman-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/oman-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/oman-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/oman-air/page/10/
10/50 pages extraction completed for oman-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/oman-air/page/11/
Scraping page 12: https://www.airlinequ

101. onur-air

In [ ]:
scrape_airline_reviews("onur-air", 14)


SCRAPING ONUR-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/onur-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/onur-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/onur-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/onur-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/onur-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/onur-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/onur-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/onur-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/onur-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/onur-air/page/10/
10/14 pages extraction completed for onur-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/onur-air/page/11/
Scraping page 12: https://www.airlinequ

In [ ]:
O_airlines_review = pd.concat([
 oman_air_reviews,
 onur_air_reviews
], axis=0, ignore_index=True)

In [ ]:
O_airlines_review.shape

(626, 16)

In [ ]:
O_airlines_review.to_csv('O_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH P**

102. pia-pakistan-international-airlines

In [ ]:
scrape_airline_reviews("pia-pakistan-international-airlines", 17)


SCRAPING PIA-PAKISTAN-INTERNATIONAL-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/pia-pakistan-international-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/pia-pakistan-internationa

103. pegasus-airlines

In [ ]:
scrape_airline_reviews("pegasus-airlines", 46)


SCRAPING PEGASUS-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/pegasus-airlines/page/10/
10/46 pages extraction completed for pegasus-airlines
Scraping page 11: https://

104. philippine-airlines

In [ ]:
scrape_airline_reviews("philippine-airlines", 94)


SCRAPING PHILIPPINE-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/philippine-airlines/page/10/
10/94 pages extraction completed for philippine

105. play

In [ ]:
scrape_airline_reviews("play", 23)


SCRAPING PLAY

Scraping page 1: https://www.airlinequality.com/airline-reviews/play/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/play/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/play/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/play/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/play/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/play/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/play/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/play/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/play/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/play/page/10/
10/23 pages extraction completed for play
Scraping page 11: https://www.airlinequality.com/airline-reviews/play/page/11/
Scraping page 12: https://www.airlinequality.com/airline-reviews/play/page/12/
Scraping pag

106. porter-airlines

In [ ]:
scrape_airline_reviews("porter-airlines", 42)


SCRAPING PORTER-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/porter-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/porter-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/porter-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/porter-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/porter-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/porter-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/porter-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/porter-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/porter-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/porter-airlines/page/10/
10/42 pages extraction completed for porter-airlines
Scraping page 11: https://www.airlineq

107. precision

In [ ]:
scrape_airline_reviews("precision", 11)


SCRAPING PRECISION

Scraping page 1: https://www.airlinequality.com/airline-reviews/precision/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/precision/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/precision/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/precision/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/precision/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/precision/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/precision/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/precision/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/precision/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/precision/page/10/
10/11 pages extraction completed for precision
Scraping page 11: https://www.airlinequality.com/airline-reviews/precision/page/11/
11/11 pages extraction com

In [ ]:
precision_reviews.head()

,Airline,Reviewer Names,Review Date,Month Flown,Review Text,Type of Traveller,Seat Type,Route,Seat Comfort,Cabin Staff Service,Food & Beverages,Inflight Entertainment,Value For Money,Ground Service,Wifi & Connectivity,Recommended
0,precision,Georgios Vrontos,2025-02-03,January 2025,"✅ Trip Verified | From Zanzibar to Arusha, A...",Couple Leisure,Economy Class,Zanzibar to Arusha,5,5,5,N/A,5,5,N/A,Yes
1,precision,Said Ali,2024-10-14,October 2024,✅ Trip Verified | I recently had an extremel...,Couple Leisure,Economy Class,Kilimanjaro to Dar es Salaam Airport,1,1,1,1,1,1,1,No
2,precision,Damien Lewis,2024-08-27,August 2024,Not Verified | World’s worst airline. Do not ...,Family Leisure,Economy Class,Dar-es-salaam to Arusha,1,N/A,N/A,N/A,1,1,N/A,No
3,precision,Karim Ismail,2024-06-19,June 2024,✅ Trip Verified | After booking with them I we...,Couple Leisure,Economy Class,Dar es Salaam to Zanzibar,3,5,N/A,N/A,5,5,N/A,Yes
4,precision,Muhumed Diriye,2024-05-24,May 2024,✅ Trip Verified | I flew with Precision Air t...,Solo Leisure,Economy Class,Nairobi to Moroni via Dar Es Salaam,1,1,1,1,1,1,1,No


In [ ]:
P_airlines_review = pd.concat([
 pegasus_airlines_reviews,
 philippine_airlines_reviews,
 play_reviews,
 porter_airlines_reviews,
 precision_reviews
], axis=0, ignore_index=True)

In [ ]:
P_airlines_review.shape

(2136, 16)

In [ ]:
P_airlines_review.to_csv('P_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH R**

108. royal-air-maroc

In [ ]:
scrape_airline_reviews("royal-air-maroc", 37)


SCRAPING ROYAL-AIR-MAROC

Scraping page 1: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/royal-air-maroc/page/10/
10/37 pages extraction completed for royal-air-maroc
Scraping page 11: https://www.airlineq

109. royal-brunei-airlines

In [ ]:
scrape_airline_reviews("royal-brunei-airlines", 38)


SCRAPING ROYAL-BRUNEI-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/royal-brunei-airlines/page/10/
10/38 pages extraction co

110. royal-jordanian-airlines

In [ ]:
scrape_airline_reviews("royal-jordanian-airlines", 39)


SCRAPING ROYAL-JORDANIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/royal-jordanian-airlines/p

111. rwandair

In [ ]:
scrape_airline_reviews("rwandair", 13)


SCRAPING RWANDAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/rwandair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/rwandair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/rwandair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/rwandair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/rwandair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/rwandair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/rwandair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/rwandair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/rwandair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/rwandair/page/10/
10/13 pages extraction completed for rwandair
Scraping page 11: https://www.airlinequality.com/airline-reviews/rwandair/page/11/
Scraping page 12: https://www.airlinequ

112. ryanair

In [ ]:
scrape_airline_reviews("ryanair", 233)


SCRAPING RYANAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/ryanair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/ryanair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/ryanair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/ryanair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/ryanair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/ryanair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/ryanair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/ryanair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/ryanair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/ryanair/page/10/
10/233 pages extraction completed for ryanair
Scraping page 11: https://www.airlinequality.com/airline-reviews/ryanair/page/11/
Scraping page 12: https://www.airlinequality.com/ai

In [ ]:
R_airlines_review = pd.concat([
 royal_air_maroc_reviews,
 royal_brunei_airlines_reviews,
 royal_jordanian_airlines_reviews,
 rwandair_reviews,
 ryanair_reviews
], axis=0, ignore_index=True)

In [ ]:
R_airlines_review.shape

(3571, 16)

In [ ]:
R_airlines_review.to_csv('R_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH S**

113. sas-scandinavian-airlines

In [ ]:
scrape_airline_reviews("sas-scandinavian-airlines", 92)


SCRAPING SAS-SCANDINAVIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/sas-scandinavian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/sas-scandinavian

114. sata-air-azores

In [ ]:
scrape_airline_reviews("sata-air-azores", 18)


SCRAPING SATA-AIR-AZORES

Scraping page 1: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/sata-air-azores/page/10/
10/18 pages extraction completed for sata-air-azores
Scraping page 11: https://www.airlineq

115. sata-internacional

In [ ]:
scrape_airline_reviews("sata-internacional", 11)


SCRAPING SATA-INTERNACIONAL

Scraping page 1: https://www.airlinequality.com/airline-reviews/sata-internacional/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/sata-internacional/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/sata-internacional/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/sata-internacional/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/sata-internacional/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/sata-internacional/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/sata-internacional/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/sata-internacional/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/sata-internacional/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/sata-internacional/page/10/
10/11 pages extraction completed for sata-internacional
Sc

116. saudi-arabian-airlines

In [ ]:
scrape_airline_reviews("saudi-arabian-airlines", 47)


SCRAPING SAUDI-ARABIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/saudi-arabian-airlines/page/10/
10/47 pages ex

117. scoot

In [ ]:
scrape_airline_reviews("scoot", 72)


SCRAPING SCOOT

Scraping page 1: https://www.airlinequality.com/airline-reviews/scoot/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/scoot/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/scoot/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/scoot/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/scoot/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/scoot/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/scoot/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/scoot/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/scoot/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/scoot/page/10/
10/72 pages extraction completed for scoot
Scraping page 11: https://www.airlinequality.com/airline-reviews/scoot/page/11/
Scraping page 12: https://www.airlinequality.com/airline-reviews/scoot/page/12

118. silkair

In [ ]:
scrape_airline_reviews("silkair", 22)


SCRAPING SILKAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/silkair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/silkair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/silkair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/silkair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/silkair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/silkair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/silkair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/silkair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/silkair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/silkair/page/10/
10/22 pages extraction completed for silkair
Scraping page 11: https://www.airlinequality.com/airline-reviews/silkair/page/11/
Scraping page 12: https://www.airlinequality.com/air

119. singapore-airlines

In [ ]:
scrape_airline_reviews("singapore-airlines", 164)


SCRAPING SINGAPORE-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/singapore-airlines/page/10/
10/164 pages extraction completed for singapore-airlines
S

120. sky-express-airlines

In [ ]:
scrape_airline_reviews("sky-express-airlines", 11)


SCRAPING SKY-EXPRESS-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/sky-express-airlines/page/10/
10/11 pages extraction completed for

121. silver-airways

In [ ]:
scrape_airline_reviews("silver-airways", 22)


SCRAPING SILVER-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/silver-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/silver-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/silver-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/silver-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/silver-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/silver-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/silver-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/silver-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/silver-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/silver-airways/page/10/
10/22 pages extraction completed for silver-airways
Scraping page 11: https://www.airlinequality.com/a

122. smartwings

In [ ]:
scrape_airline_reviews("smartwings", 11)


SCRAPING SMARTWINGS

Scraping page 1: https://www.airlinequality.com/airline-reviews/smartwings/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/smartwings/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/smartwings/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/smartwings/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/smartwings/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/smartwings/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/smartwings/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/smartwings/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/smartwings/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/smartwings/page/10/
10/11 pages extraction completed for smartwings
Scraping page 11: https://www.airlinequality.com/airline-reviews/smartwings/page/11/
11/11 pages e

123. south-african-airways

In [ ]:
scrape_airline_reviews("south-african-airways", 44)


SCRAPING SOUTH-AFRICAN-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/south-african-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/south-african-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/south-african-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/south-african-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/south-african-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/south-african-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/south-african-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/south-african-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/south-african-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/south-african-airways/page/10/
10/44 pages extraction co

124. southwest-airlines

In [ ]:
scrape_airline_reviews("southwest-airlines", 188)


SCRAPING SOUTHWEST-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/southwest-airlines/page/10/
10/188 pages extraction completed for southwest-airlines
S

125. spicejet

In [ ]:
scrape_airline_reviews("spicejet", 67)


SCRAPING SPICEJET

Scraping page 1: https://www.airlinequality.com/airline-reviews/spicejet/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/spicejet/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/spicejet/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/spicejet/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/spicejet/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/spicejet/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/spicejet/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/spicejet/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/spicejet/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/spicejet/page/10/
10/67 pages extraction completed for spicejet
Scraping page 11: https://www.airlinequality.com/airline-reviews/spicejet/page/11/
Scraping page 12: https://www.airlinequ

126. srilankan-airlines

In [ ]:
scrape_airline_reviews("srilankan-airlines", 68)


SCRAPING SRILANKAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/srilankan-airlines/page/10/
10/68 pages extraction completed for srilankan-airlines
Sc

127. sun-country-airlines

In [ ]:
scrape_airline_reviews("sun-country-airlines", 37)


SCRAPING SUN-COUNTRY-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/sun-country-airlines/page/10/
10/37 pages extraction completed for

128. sunwing-airlines

In [ ]:
scrape_airline_reviews("sunwing-airlines", 73)


SCRAPING SUNWING-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/sunwing-airlines/page/10/
10/73 pages extraction completed for sunwing-airlines
Scraping page 11: https://

129. swiss-international-air-lines

In [ ]:
scrape_airline_reviews("swiss-international-air-lines", 112)


SCRAPING SWISS-INTERNATIONAL-AIR-LINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/swiss-international-air-lines/page/9/
Scraping page 10: https://www.airlinequal

130. swoop

In [ ]:
scrape_airline_reviews("swoop", 73)


SCRAPING SWOOP

Scraping page 1: https://www.airlinequality.com/airline-reviews/swoop/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/swoop/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/swoop/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/swoop/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/swoop/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/swoop/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/swoop/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/swoop/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/swoop/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/swoop/page/10/
10/73 pages extraction completed for swoop
Scraping page 11: https://www.airlinequality.com/airline-reviews/swoop/page/11/
Scraping page 12: https://www.airlinequality.com/airline-reviews/swoop/page/12

In [ ]:
S_airlines_review = pd.concat([
 sas_scandinavian_airlines_reviews,
 sata_air_azores_reviews,
 sata_internacional_reviews,
 saudi_arabian_airlines_reviews,
 scoot_reviews,
 silkair_reviews,
 singapore_airlines_reviews,
 sky_express_airlines_reviews,
 silver_airways_reviews,
 smartwings_reviews,
 south_african_airways_reviews,
 southwest_airlines_reviews,
 spicejet_reviews,
 srilankan_airlines_reviews,
 sun_country_airlines_reviews,
 sunwing_airlines_reviews,
 swiss_international_air_lines_reviews,
 swoop_reviews
], axis=0, ignore_index=True)

In [ ]:
S_airlines_review.shape

(11266, 16)

In [ ]:
S_airlines_review.to_csv('S_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH T**

131. tam-airlines

In [ ]:
scrape_airline_reviews("tam-airlines", 23)


SCRAPING TAM-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/tam-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/tam-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/tam-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/tam-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/tam-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/tam-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/tam-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/tam-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/tam-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/tam-airlines/page/10/
10/23 pages extraction completed for tam-airlines
Scraping page 11: https://www.airlinequality.com/airline-reviews/tam-airli

132. tap-portugal

In [ ]:
scrape_airline_reviews("tap-portugal", 157)


SCRAPING TAP-PORTUGAL

Scraping page 1: https://www.airlinequality.com/airline-reviews/tap-portugal/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/tap-portugal/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/tap-portugal/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/tap-portugal/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/tap-portugal/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/tap-portugal/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/tap-portugal/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/tap-portugal/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/tap-portugal/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/tap-portugal/page/10/
10/157 pages extraction completed for tap-portugal
Scraping page 11: https://www.airlinequality.com/airline-reviews/tap-port

133. thai-airways

In [ ]:
scrape_airline_reviews("thai-airways", 108)


SCRAPING THAI-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/thai-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/thai-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/thai-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/thai-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/thai-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/thai-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/thai-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/thai-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/thai-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/thai-airways/page/10/
10/108 pages extraction completed for thai-airways
Scraping page 11: https://www.airlinequality.com/airline-reviews/thai-air

134. thai-airasia

In [ ]:
scrape_airline_reviews("thai-airasia", 14)


SCRAPING THAI-AIRASIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/thai-airasia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/thai-airasia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/thai-airasia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/thai-airasia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/thai-airasia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/thai-airasia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/thai-airasia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/thai-airasia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/thai-airasia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/thai-airasia/page/10/
10/14 pages extraction completed for thai-airasia
Scraping page 11: https://www.airlinequality.com/airline-reviews/thai-aira

135. tarom-romanian-airlines

In [ ]:
scrape_airline_reviews("tarom-romanian-airlines", 20)


SCRAPING TAROM-ROMANIAN-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/tarom-romanian-airlines/page/10/
10/

136. thai-lion-air

In [ ]:
scrape_airline_reviews("thai-lion-air", 18)


SCRAPING THAI-LION-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/thai-lion-air/page/10/
10/18 pages extraction completed for thai-lion-air
Scraping page 11: https://www.airlinequality.com/airline-revie

137. thai-smile-airways

In [ ]:
scrape_airline_reviews("thai-smile-airways", 28)


SCRAPING THAI-SMILE-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/thai-smile-airways/page/10/
10/28 pages extraction completed for thai-smile-airways
Sc

138. thomas-cook-airlines

In [ ]:
scrape_airline_reviews("thomas-cook-airlines", 65)


SCRAPING THOMAS-COOK-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/thomas-cook-airlines/page/10/
10/65 pages extraction completed for

139. thomson-airways

In [ ]:
scrape_airline_reviews("thomson-airways", 59)


SCRAPING THOMSON-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/thomson-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/thomson-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/thomson-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/thomson-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/thomson-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/thomson-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/thomson-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/thomson-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/thomson-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/thomson-airways/page/10/
10/59 pages extraction completed for thomson-airways
Scraping page 11: https://www.airlineq

140. tigerair

In [ ]:
scrape_airline_reviews("tigerair", 19)


SCRAPING TIGERAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/tigerair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/tigerair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/tigerair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/tigerair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/tigerair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/tigerair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/tigerair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/tigerair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/tigerair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/tigerair/page/10/
10/19 pages extraction completed for tigerair
Scraping page 11: https://www.airlinequality.com/airline-reviews/tigerair/page/11/
Scraping page 12: https://www.airlinequ

141. tigerair-australia

In [ ]:
scrape_airline_reviews("tigerair-australia", 51)


SCRAPING TIGERAIR-AUSTRALIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/tigerair-australia/page/10/
10/51 pages extraction completed for tigerair-australia
Sc

142. transavia

In [ ]:
scrape_airline_reviews("transavia", 28)


SCRAPING TRANSAVIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/transavia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/transavia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/transavia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/transavia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/transavia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/transavia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/transavia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/transavia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/transavia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/transavia/page/10/
10/28 pages extraction completed for transavia
Scraping page 11: https://www.airlinequality.com/airline-reviews/transavia/page/11/
Scraping page 12: https://

In [ ]:
T_airlines_review = pd.concat([
 tam_airlines_reviews,
 tap_portugal_reviews,
 thai_airways_reviews,
 thai_airasia_reviews,
 tarom_romanian_airlines_reviews,
 thai_lion_air_reviews,
 thai_smile_airways_reviews,
 thomas_cook_airlines_reviews,
 thomson_airways_reviews,
 tigerair_reviews,
 tigerair_australia_reviews,
 transavia_reviews
], axis=0, ignore_index=True)

In [ ]:
T_airlines_review.shape

(5847, 16)

In [ ]:
T_airlines_review.to_csv('T_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH U**

143. ukraine-international-airlines

In [ ]:
scrape_airline_reviews("ukraine-international-airlines", 40)


SCRAPING UKRAINE-INTERNATIONAL-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/ukraine-international-airlines/page/9/
Scraping page 10: https://www.a

144. us-airways

In [ ]:
scrape_airline_reviews("us-airways", 46)


SCRAPING US-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/us-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/us-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/us-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/us-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/us-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/us-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/us-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/us-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/us-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/us-airways/page/10/
10/46 pages extraction completed for us-airways
Scraping page 11: https://www.airlinequality.com/airline-reviews/us-airways/page/11/
Scraping page

In [ ]:
U_airlines_review = pd.concat([
 ukraine_international_airlines_reviews,
 us_airways_reviews
], axis=0, ignore_index=True)

In [ ]:
U_airlines_review.shape

(851, 16)

In [ ]:
U_airlines_review.to_csv('U_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH V**

145. vietjetair

In [ ]:
scrape_airline_reviews("vietjetair", 72)


SCRAPING VIETJETAIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/vietjetair/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/vietjetair/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/vietjetair/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/vietjetair/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/vietjetair/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/vietjetair/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/vietjetair/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/vietjetair/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/vietjetair/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/vietjetair/page/10/
10/72 pages extraction completed for vietjetair
Scraping page 11: https://www.airlinequality.com/airline-reviews/vietjetair/page/11/
Scraping page

146. vietnam-airlines

In [ ]:
scrape_airline_reviews("vietnam-airlines", 76)


SCRAPING VIETNAM-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/vietnam-airlines/page/10/
10/76 pages extraction completed for vietnam-airlines
Scraping page 11: https://

147. virgin-america

In [ ]:
scrape_airline_reviews("virgin-america", 31)


SCRAPING VIRGIN-AMERICA

Scraping page 1: https://www.airlinequality.com/airline-reviews/virgin-america/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/virgin-america/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/virgin-america/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/virgin-america/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/virgin-america/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/virgin-america/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/virgin-america/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/virgin-america/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/virgin-america/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/virgin-america/page/10/
10/31 pages extraction completed for virgin-america
Scraping page 11: https://www.airlinequality.com/a

148. virgin-atlantic-airways

In [ ]:
scrape_airline_reviews("virgin-atlantic-airways", 87)


SCRAPING VIRGIN-ATLANTIC-AIRWAYS

Scraping page 1: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/virgin-atlantic-airways/page/10/
10/

149. virgin-australia

In [ ]:
scrape_airline_reviews("virgin-australia", 105)


SCRAPING VIRGIN-AUSTRALIA

Scraping page 1: https://www.airlinequality.com/airline-reviews/virgin-australia/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/virgin-australia/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/virgin-australia/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/virgin-australia/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/virgin-australia/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/virgin-australia/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/virgin-australia/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/virgin-australia/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/virgin-australia/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/virgin-australia/page/10/
10/105 pages extraction completed for virgin-australia
Scraping page 11: https:/

150. vistara

In [ ]:
scrape_airline_reviews("vistara", 28)


SCRAPING VISTARA

Scraping page 1: https://www.airlinequality.com/airline-reviews/vistara/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/vistara/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/vistara/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/vistara/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/vistara/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/vistara/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/vistara/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/vistara/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/vistara/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/vistara/page/10/
10/28 pages extraction completed for vistara
Scraping page 11: https://www.airlinequality.com/airline-reviews/vistara/page/11/
Scraping page 12: https://www.airlinequality.com/air

151. vivaaerobus

In [ ]:
scrape_airline_reviews("vivaaerobus", 16)


SCRAPING VIVAAEROBUS

Scraping page 1: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/10/
10/16 pages extraction completed for vivaaerobus
Scraping page 11: https://www.airlinequality.com/airline-reviews/vivaaerobus/page/11/


152. volaris

In [ ]:
scrape_airline_reviews("volaris", 42)


SCRAPING VOLARIS

Scraping page 1: https://www.airlinequality.com/airline-reviews/volaris/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/volaris/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/volaris/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/volaris/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/volaris/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/volaris/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/volaris/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/volaris/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/volaris/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/volaris/page/10/
10/42 pages extraction completed for volaris
Scraping page 11: https://www.airlinequality.com/airline-reviews/volaris/page/11/
Scraping page 12: https://www.airlinequality.com/air

153. volotea

In [ ]:
scrape_airline_reviews("volotea", 58)


SCRAPING VOLOTEA

Scraping page 1: https://www.airlinequality.com/airline-reviews/volotea/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/volotea/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/volotea/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/volotea/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/volotea/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/volotea/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/volotea/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/volotea/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/volotea/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/volotea/page/10/
10/58 pages extraction completed for volotea
Scraping page 11: https://www.airlinequality.com/airline-reviews/volotea/page/11/
Scraping page 12: https://www.airlinequality.com/air

154. vueling-airlines

In [ ]:
scrape_airline_reviews("vueling-airlines", 125)


SCRAPING VUELING-AIRLINES

Scraping page 1: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/vueling-airlines/page/10/
10/125 pages extraction completed for vueling-airlines
Scraping page 11: https:/

In [ ]:
V_airlines_review = pd.concat([
 vietjetair_reviews,
 vietnam_airlines_reviews,
 virgin_america_reviews,
 virgin_atlantic_airways_reviews,
 virgin_australia_reviews,
 vistara_reviews,
 vivaaerobus_reviews,
 volaris_reviews,
 volotea_reviews,
 vueling_airlines_reviews
], axis=0, ignore_index=True)

In [ ]:
V_airlines_review.shape

(6351, 16)

In [ ]:
V_airlines_review.to_csv('V_airlines_review.csv', index=False)

# **AIRLINES STARTING WITH W**

155. westjet

In [ ]:
scrape_airline_reviews("westjet", 117)


SCRAPING WESTJET

Scraping page 1: https://www.airlinequality.com/airline-reviews/westjet/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/westjet/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/westjet/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/westjet/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/westjet/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/westjet/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/westjet/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/westjet/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/westjet/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/westjet/page/10/
10/117 pages extraction completed for westjet
Scraping page 11: https://www.airlinequality.com/airline-reviews/westjet/page/11/
Scraping page 12: https://www.airlinequality.com/ai

116. wizz-air

In [ ]:
scrape_airline_reviews("wizz-air", 153)


SCRAPING WIZZ-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/wizz-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/wizz-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/wizz-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/wizz-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/wizz-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/wizz-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/wizz-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/wizz-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/wizz-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/wizz-air/page/10/
10/153 pages extraction completed for wizz-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/wizz-air/page/11/
Scraping page 12: https://www.airlineq

117. wow-air

In [ ]:
scrape_airline_reviews("wow-air", 58)


SCRAPING WOW-AIR

Scraping page 1: https://www.airlinequality.com/airline-reviews/wow-air/page/1/
Scraping page 2: https://www.airlinequality.com/airline-reviews/wow-air/page/2/
Scraping page 3: https://www.airlinequality.com/airline-reviews/wow-air/page/3/
Scraping page 4: https://www.airlinequality.com/airline-reviews/wow-air/page/4/
Scraping page 5: https://www.airlinequality.com/airline-reviews/wow-air/page/5/
Scraping page 6: https://www.airlinequality.com/airline-reviews/wow-air/page/6/
Scraping page 7: https://www.airlinequality.com/airline-reviews/wow-air/page/7/
Scraping page 8: https://www.airlinequality.com/airline-reviews/wow-air/page/8/
Scraping page 9: https://www.airlinequality.com/airline-reviews/wow-air/page/9/
Scraping page 10: https://www.airlinequality.com/airline-reviews/wow-air/page/10/
10/58 pages extraction completed for wow-air
Scraping page 11: https://www.airlinequality.com/airline-reviews/wow-air/page/11/
Scraping page 12: https://www.airlinequality.com/air

In [ ]:
W_airlines_review = pd.concat([
 westjet_reviews,
 wizz_air_reviews,
 wow_air_reviews
], axis=0, ignore_index=True)

In [ ]:
W_airlines_review.shape

(3269, 16)

In [ ]:
W_airlines_review.to_csv('W_airlines_review.csv', index=False)

In [ ]:
master_data = pd.concat([
    A_airlines_review,
    B_airlines_review,
    C_airlines_review,
    D_airlines_review,
    E_airlines_review,
    F_airlines_review,
    G_airlines_review,
    H_airlines_review,
    I_airlines_review,
    J_airlines_review,
    K_airlines_review,
    L_airlines_review,
    M_airlines_review,
    N_airlines_review,
    O_airlines_review,
    P_airlines_review,
    R_airlines_review,
    S_airlines_review,
    T_airlines_review,
    U_airlines_review,
    V_airlines_review,
    W_airlines_review
], axis=0, ignore_index=True)

In [ ]:
master_data.shape

(80221, 16)

In [ ]:
master_data.to_csv('master_data.csv', index=False)

In [ ]:
df = pd.concat([
 master_data,
  abc
], axis=0, ignore_index=True)

In [ ]:
abc=pd.read_csv("/content/airlines_review.csv")

In [ ]:
df.shape

(127987, 16)

In [ ]:
df.to_csv('df.csv', index=False)

In [ ]:
df.isna().sum()

,0
Airline,0
Reviewer Names,0
Review Date,0
Month Flown,8467
Review Text,0
Type of Traveller,8414
Seat Type,891
Route,8610
Seat Comfort,2383
Cabin Staff Service,2538


In [ ]:
df2=df.dropna()

In [ ]:
df.s

In [ ]:
df2.shape

(94094, 16)